In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, SimpleRNN, TimeDistributed, LSTM, GRU, Bidirectional
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [2]:
Data = np.load('DATA/EpilepsyData.npz', allow_pickle=True)

In [3]:
Train_x = Data['Train_x']
Train_y = Data['Train_y']
Test_x = Data['Test_x']
Test_y = Data['Test_y']

### Data Shuffle

In [5]:
arr = np.arange(Train_x.shape[0])
np.random.shuffle(arr)
arr

array([ 75,  59,  26,  12,  65,  79,  49, 133,  30, 105,   1,  24, 136,
        80,  78,  66, 118,  20, 109, 106,  25,  88,  36,  13, 120,  87,
        71,  42,  19, 135, 112,  34, 100,  51,  97, 124, 126,  60,  57,
       122,  41, 101,  52,  76,  89,  16, 134, 113,  95,  54,  67, 121,
        50,  96, 131, 125, 107, 130,  68, 102,   3,   6, 111,  83,  48,
        15,  39,  38, 117,  22,  92, 104,  74, 103,  94,  56,  37,   5,
         9,  53,  32,  14,  99,  35, 132, 114,  31,  91,  18,  45,  61,
        47,  43,  77, 129,  10,  27,  84,  72,  46,  82, 123, 108,  58,
       119,  28,  40,  33,  21,   0,  93,  55,  98,  73, 127,  29,  11,
        69,  62,  17,  85, 110, 116,  86,  23,   8, 128,   7, 115,   4,
        81,  90,  63,  64,  44,   2,  70])

In [6]:
Train_x = Train_x[arr]
Train_y = Train_y[arr]

In [7]:
arr = np.arange(Test_x.shape[0])
np.random.shuffle(arr)

Test_x = Test_x[arr]
Test_y = Test_y[arr]

In [9]:
Test_x.shape

(138, 206, 3)

In [12]:
n_feature_maps = 64

In [19]:
input_layer = tf.keras.layers.Input( (206, 3, ) )

conv_x = tf.keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
conv_x = tf.keras.layers.BatchNormalization()(conv_x)
conv_x = tf.keras.layers.Activation('relu')(conv_x)

In [20]:
model = tf.keras.models.Model(inputs=input_layer, outputs=conv_x)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 206, 3)]          0         
                                                                 
 conv1d_8 (Conv1D)           (None, 206, 64)           1600      
                                                                 
 batch_normalization_4 (Batc  (None, 206, 64)          256       
 hNormalization)                                                 
                                                                 
 activation_3 (Activation)   (None, 206, 64)           0         
                                                                 
Total params: 1,856
Trainable params: 1,728
Non-trainable params: 128
_________________________________________________________________


In [25]:
model.predict(Train_x[:1]).shape

1/1 [==============================] - 0s 12ms/step


(1, 206, 64)

In [27]:
input_layer = tf.keras.layers.Input( (206, 3, ) )

conv_x = tf.keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
conv_x = tf.keras.layers.BatchNormalization()(conv_x)
conv_x = tf.keras.layers.Activation('relu')(conv_x)
gap_layer = tf.keras.layers.GlobalAveragePooling1D()(conv_x)
output_layer = tf.keras.layers.Dense(4, activation='softmax')(gap_layer)

In [28]:
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 206, 3)]          0         
                                                                 
 conv1d_10 (Conv1D)          (None, 206, 64)           1600      
                                                                 
 batch_normalization_6 (Batc  (None, 206, 64)          256       
 hNormalization)                                                 
                                                                 
 activation_5 (Activation)   (None, 206, 64)           0         
                                                                 
 global_average_pooling1d_1   (None, 64)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense (Dense)               (None, 4)                 260 

In [29]:
model.compile(loss = "categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 206, 3)]          0         
                                                                 
 conv1d_10 (Conv1D)          (None, 206, 64)           1600      
                                                                 
 batch_normalization_6 (Batc  (None, 206, 64)          256       
 hNormalization)                                                 
                                                                 
 activation_5 (Activation)   (None, 206, 64)           0         
                                                                 
 global_average_pooling1d_1   (None, 64)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense (Dense)               (None, 4)                 260 

In [30]:
history_finetunning = model.fit(
    Train_x, Train_y,
    epochs=30,
    batch_size=52,
    #callbacks=CALLBACK,
    validation_data=(Test_x, Test_y),
    shuffle=True)

Epoch 1/30
3/3 [==============================] - 1s 77ms/step - loss: 1.5684 - accuracy: 0.2555 - val_loss: 1.3564 - val_accuracy: 0.3551
Epoch 2/30
3/3 [==============================] - 0s 18ms/step - loss: 1.4217 - accuracy: 0.4088 - val_loss: 1.3259 - val_accuracy: 0.4565
Epoch 3/30
3/3 [==============================] - 0s 17ms/step - loss: 1.2881 - accuracy: 0.5474 - val_loss: 1.2970 - val_accuracy: 0.5217
Epoch 4/30
3/3 [==============================] - 0s 16ms/step - loss: 1.1729 - accuracy: 0.6350 - val_loss: 1.2700 - val_accuracy: 0.5507
Epoch 5/30
3/3 [==============================] - 0s 17ms/step - loss: 1.0779 - accuracy: 0.6788 - val_loss: 1.2443 - val_accuracy: 0.6014
Epoch 6/30
3/3 [==============================] - 0s 19ms/step - loss: 0.9977 - accuracy: 0.6861 - val_loss: 1.2198 - val_accuracy: 0.6159
Epoch 7/30
3/3 [==============================] - 0s 20ms/step - loss: 0.9292 - accuracy: 0.7153 - val_loss: 1.1967 - val_accuracy: 0.6304
Epoch 8/30
3/3 [===========